In [1]:
from transformers import pipeline
import tensorflow as tf
import numpy as np
import os

/mnt/nrdstor/statsgeneral/gayara/tf_transformers_env_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-01 14:17:46.312061: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-01 14:17:47.533364: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-01 14:17:47.533437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been regis

In [2]:
tf.__version__

'2.15.0'

In [3]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Text classification

In [4]:
classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9598047137260437}]

In [5]:
# can pass multiple sentences too - notice how they are passed in a list
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)


[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

Zero-shot classification

In [6]:
classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445962071418762, 0.11197620630264282, 0.04342762380838394]}

In [7]:
classifier("This series was very entertaining, especially the storyline about cult-like group.", candidate_labels=["Movie", "TV show", "Book"])

{'sequence': 'This series was very entertaining, especially the storyline about cult-like group.',
 'labels': ['TV show', 'Movie', 'Book'],
 'scores': [0.9846734404563904, 0.010040625929832458, 0.005285870283842087]}

In [8]:
classifier("Could not keep it down, read page after page until I finished it", candidate_labels = ['Book', "Paper", "Cartoon"])

{'sequence': 'Could not keep it down, read page after page until I finished it',
 'labels': ['Book', 'Paper', 'Cartoon'],
 'scores': [0.7489259839057922, 0.15304717421531677, 0.09802688658237457]}

Text generation

In [9]:
generator = pipeline("text-generation")

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [10]:
generator("In this course, we will teach you how to")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to handle any number of different situations and concepts in your own life. We will also learn how to do a very basic task or concept of a mental illness.\n\nWe will also learn how to practice a few different things to help you better understand and understand that you are an ill person.\n\nIf you are interested in getting started then we are looking forward to hearing from you!\n\nIf you have any questions or if you are interested in learning more about this course please go to our website and send us a message.'}]

In [11]:
generator("In this course, we will teach you how to", num_return_sequences  = 2, max_new_tokens = 15)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to create a simple data model that creates and displays data points from a variety of'},
 {'generated_text': 'In this course, we will teach you how to build an app in JavaScript. We will then go through the steps of implementing'}]

##### Try different models (and not just the defaults)

In [12]:
generator = pipeline("text-generation", model="HuggingFaceTB/SmolLM2-360M")
generator(
    "In this course, we will teach you how to",
    max_new_tokens=30,
    num_return_sequences=2,
)

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to make your own custom website and blog, and how to take advantage of the many resources available on the Internet to create a successful online business. We will'},
 {'generated_text': 'In this course, we will teach you how to write a research paper in a way that will help you create a successful paper and convince your audience of the importance of your topic. To do that,'}]

Mask Filling

In [13]:
# Idea: Fill-in the banks of a given text

unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'score': 0.19619743525981903,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04052699729800224,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'}]

Named Entity Recognition

In [14]:
ner = pipeline("ner", grouped_entities=True)
# notice over here we pass grouped_entities = True. This will group the parts of the sentece that belong to the same entity together. In the example, hugging face is correctly identified by the model as belonging to the category ORG.
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/mnt/nr

[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [15]:
ner_2 = pipeline("ner", grouped_entities=True)
ner_2("We are Sylvain and Taylor. She and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'entity_group': 'PER',
  'score': 0.98277956,
  'word': 'Sylvain',
  'start': 7,
  'end': 14},
 {'entity_group': 'PER',
  'score': 0.9983236,
  'word': 'Taylor',
  'start': 19,
  'end': 25},
 {'entity_group': 'ORG',
  'score': 0.9254735,
  'word': 'Hugging Face',
  'start': 45,
  'end': 57},
 {'entity_group': 'LOC',
  'score': 0.9930924,
  'word': 'Brooklyn',
  'start': 61,
  'end': 69}]

Question Answering

In [16]:
question_answer = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [17]:
question_answer(question = "Where do I work?", context = "My name is Gayara and I am a graduate student at UNL.")

{'score': 0.9414548873901367, 'start': 49, 'end': 52, 'answer': 'UNL'}

In [18]:
# Note this pipeline just extract information, and does not generate an answer

Text summarization

In [19]:
summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil,    electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering .'}]